In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'sarfusion'))
sys.path.append(str(Path.cwd().parent / 'sarfusion' / 'data'))

In [1]:
import pandas as pd 
import wandb
import sys
api = wandb.Api()

In [2]:
def linearize_dict(d, parent_key='', sep='.'):
    """
    Linearize a nested dictionary.
    
    Args:
        d (dict): The dictionary to linearize.
        parent_key (str): The base key for recursion, used to build the new keys.
        sep (str): The separator for concatenating keys.
        
    Returns:
        dict: A flattened dictionary with concatenated keys.
    """
    items = []
    for key, value in d.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(linearize_dict(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)

In [3]:
# Project is specified by <entity/project-name>
runs = api.runs("pasqualedem/SARTFusion")

runs_params = []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    run_params = run.summary._json_dict

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    run_params = {
        **run_params,
        **{k: v for k,v in run.config.items()
          if not k.startswith('_')}
    }
        

    # .name is the human-readable name of the run.
    run_params["name"] = run.name
    run_params = linearize_dict(run_params)

    runs_params.append(run_params)

runs_df = pd.DataFrame(runs_params)

In [4]:
runs_df.columns

Index(['_runtime', '_step', '_timestamp', '_wandb.runtime', 'test/classes',
       'test/loss', 'test/map', 'test/map_50', 'test/map_75', 'test/map_large',
       'test/map_medium', 'test/map_per_class', 'test/map_small', 'test/mar_1',
       'test/mar_10', 'test/mar_100', 'test/mar_100_per_class',
       'test/mar_large', 'test/mar_medium', 'test/mar_small', 'test/step',
       'test_predictions._latest_artifact_path', 'test_predictions._type',
       'test_predictions.artifact_path', 'test_predictions.ncols',
       'test_predictions.nrows', 'test_predictions.path',
       'test_predictions.sha256', 'test_predictions.size', 'seed', 'task',
       'model.name', 'model.params.id2label.0', 'model.params.threshold',
       'model.params.pretrained_path', 'dataset.name', 'dataset.root',
       'dataset.folders', 'dataset.preprocessor.path', 'dataset.single_class',
       'tracker.tags', 'tracker.ignored_files',
       'tracker.val_image_log_frequency', 'dataloader.batch_size',
       'dat

In [5]:
pretrained_map = {
    'checkpoints/fusiondetr_all_tepn3c4l.safetensors': "$D$",
    'checkpoints/fusiondetr_ir_00aprtfc.safetensors': "$D_{IR_o}$",
    'checkpoints/fusiondetr_vis-a-ir-s_pffks3s9.safetensors': " $D_{RGB_a-IR_s}$",
    'checkpoints/fusiondetr_vis-ir_2dm72jvg.safetensors': "$D_{RGB-IR}$",
    'checkpoints/fusiondetr_vis-s-ir-a_5jkopgg6.safetensors': "$D_{RGB_s-IR_a}$",
    'checkpoints/fusiondetr_vis_b80foknc.safetensors': "$D_{RGB_o}$"
    }

folders_map = {
    "ir": "$D_{IR_o}$",
    "vis": "$D_{RGB_o}$",
    "all": "$D$"
}

column_map = {
    "model.params.pretrained_path": "Trained on",
    "dataset.folders": "Tested on",
    "model.params.threshold": "Threshold",
    "test/map_50": "mAP@50"
}

In [7]:
tag = "Test"

filtered_df = runs_df[runs_df["tracker.tags"].apply(lambda x: isinstance(x, list) and tag in x)]
filtered_df = filtered_df[filtered_df["model.params.threshold"] == 0.5] 


columns = ["model.params.pretrained_path", 'dataset.folders', 'test/map_50']

# filtered_df[columns].groupby(["model.params.threshold"])['test/map_50'].mean()

# Apply mapping to dataframe
filtered_df["model.params.pretrained_path"] = filtered_df["model.params.pretrained_path"].map(pretrained_map)
filtered_df["dataset.folders"] = filtered_df["dataset.folders"].map(folders_map)
filtered_df["test/map_50"] = filtered_df["test/map_50"].round(3)

# Sort by dataset.folders and then by model.params.pretrained_path
filtered_df = filtered_df.sort_values(["dataset.folders", "model.params.pretrained_path"])

filtered_df = filtered_df[columns]

# Apply renaming dataframe
final_df = filtered_df.rename(column_map, axis=1)

final_df

,Trained on,Tested on,mAP@50
27,$D_{RGB_a-IR_s}$,$D$,0.263
39,$D$,$D$,0.372
21,$D_{RGB-IR}$,$D$,0.151
15,$D_{RGB_s-IR_a}$,$D$,0.249
29,$D_{RGB_a-IR_s}$,$D_{IR_o}$,0.000
41,$D$,$D_{IR_o}$,0.233
4,$D_{IR_o}$,$D_{IR_o}$,0.427
22,$D_{RGB-IR}$,$D_{IR_o}$,0.000
16,$D_{RGB_s-IR_a}$,$D_{IR_o}$,0.292
31,$D_{RGB_a-IR_s}$,$D_{RGB_o}$,0.442


In [8]:
final_df.to_csv(sys.stdout, index=False)

Trained on,Tested on,mAP@50
 $D_{RGB_a-IR_s}$,$D$,0.263
$D$,$D$,0.372
$D_{RGB-IR}$,$D$,0.151
$D_{RGB_s-IR_a}$,$D$,0.249
 $D_{RGB_a-IR_s}$,$D_{IR_o}$,0.0
$D$,$D_{IR_o}$,0.233
$D_{IR_o}$,$D_{IR_o}$,0.427
$D_{RGB-IR}$,$D_{IR_o}$,0.0
$D_{RGB_s-IR_a}$,$D_{IR_o}$,0.292
 $D_{RGB_a-IR_s}$,$D_{RGB_o}$,0.442
$D$,$D_{RGB_o}$,0.465
$D_{RGB-IR}$,$D_{RGB_o}$,0.281
$D_{RGB_o}$,$D_{RGB_o}$,0.432
$D_{RGB_s-IR_a}$,$D_{RGB_o}$,0.188


In [74]:
latex_code = final_df.to_latex(index=False, escape=False, column_format='|c|c|c|c|')
print(latex_code)

\begin{tabular}{|c|c|c|c|}
\toprule
Trained on & Threshold & Tested on & mAP@50 \\
\midrule
$D_{IR_o}$ & 0.500000 & $D_{IR_o}$ & 0.427000 \\
$D_{RGB_o}$ & 0.500000 & $D_{RGB_o}$ & 0.432000 \\
$D_{RGB_s-IR_a}$ & 0.500000 & $D$ & 0.249000 \\
$D_{RGB_s-IR_a}$ & 0.500000 & $D_{IR_o}$ & 0.292000 \\
$D_{RGB_s-IR_a}$ & 0.500000 & $D_{RGB_o}$ & 0.188000 \\
$D_{RGB-IR}$ & 0.500000 & $D$ & 0.151000 \\
$D_{RGB-IR}$ & 0.500000 & $D_{IR_o}$ & 0.000000 \\
$D_{RGB-IR}$ & 0.500000 & $D_{RGB_o}$ & 0.281000 \\
 $D_{RGB_a-IR_s}$ & 0.500000 & $D$ & 0.263000 \\
 $D_{RGB_a-IR_s}$ & 0.500000 & $D_{IR_o}$ & 0.000000 \\
 $D_{RGB_a-IR_s}$ & 0.500000 & $D_{RGB_o}$ & 0.442000 \\
$D$ & 0.500000 & $D$ & 0.372000 \\
$D$ & 0.500000 & $D_{IR_o}$ & 0.233000 \\
$D$ & 0.500000 & $D_{RGB_o}$ & 0.465000 \\
\bottomrule
\end{tabular}

